In [22]:
import numpy as np
import pandas as pd
import os
import mvpa_base_functions as bf
import pickle
from scipy import stats
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [9]:
phase = 'cond'
mask = 'fearnet'
trial_block = 1

In [20]:
dtfile = f'./sample_data/discovery_{phase}_{mask}_{trial_block}block.pkl'
svfile = f'./sample_results/predict_pattern_{phase}_{mask}_{trial_block}block.pkl'
D = pickle.load(open(dtfile, 'rb'))
csm_data = D['csm_data']
csp_data = D['csp_data']

subj_num, feat_num = csm_data.shape


param_num = 20
param_list = np.logspace(-4, 4, num=param_num, base=10)
LR = LogisticRegression(C=1, solver='liblinear')
dclf = Pipeline([('scaler', StandardScaler()), ('clf', LR)])
param_grid = {'clf__C':param_list}
clf = GridSearchCV(dclf, param_grid, n_jobs=10)

In [11]:
#bootstrap
boot_num = 10
boot_pred_patterns = np.zeros((boot_num, feat_num))
boot_cls_weights = np.zeros((boot_num, feat_num)) 
for ibt in range(boot_num):
    if ibt%10==0:
        print(ibt)
        
    ridx = np.random.choice(subj_num, subj_num)#resampling with replacement
    rX = np.concatenate((csm_data[ridx], csp_data[ridx]), axis=0)
    rY = np.concatenate((0*np.ones(subj_num),1*np.ones(subj_num)), axis=0)
    group = np.concatenate((np.arange(subj_num),np.arange(subj_num)), axis=0)
    rclf = clf
    rclf.fit(rX, rY, groups=group)
    w = rclf.best_estimator_['clf'].coef_
    pp = bf.weight_transform(StandardScaler().fit_transform(rX), w)
    
    boot_cls_weights[ibt] = w.reshape(-1)
    boot_pred_patterns[ibt] = pp.reshape(-1)

0


In [12]:
# permutation
perm_num = 10
perm_pred_patterns = np.zeros((perm_num, feat_num))
perm_cls_weights = np.zeros((perm_num, feat_num)) 

X = np.concatenate((csm_data, csp_data), axis=0)
Y = np.concatenate((0*np.ones(subj_num),1*np.ones(subj_num)), axis=0)
group = np.concatenate((np.arange(subj_num),np.arange(subj_num)), axis=0)
clf.fit(X, Y, groups=group)
best_C = clf.best_params_['clf__C']

for iperm in range(perm_num):
    if iperm%10==0:
        print(iperm)
        
    rY = bf.permute_Y(Y)
    
    bLR = LogisticRegression(C=best_C, solver='liblinear')# for permutation, use the best C to save computation time
    pclf = Pipeline([('scaler', StandardScaler()), ('clf', bLR)])
    pclf.fit(X, rY)
    
    w = pclf['clf'].coef_
    pp = bf.weight_transform(StandardScaler().fit_transform(X), w)
    
    perm_cls_weights[ibt] = w.reshape(-1)
    perm_pred_patterns[ibt] = pp.reshape(-1)

0


In [23]:
pred_patterns = np.mean(boot_pred_patterns, axis=0)
mperm_pattern = np.mean(perm_pred_patterns, axis=0, keepdims=True)
eperm_pattern = np.std(perm_pred_patterns, axis=0, keepdims=True)
zvals = (pred_patterns - mperm_pattern) / eperm_pattern
zvals = zvals.reshape(-1)
pvals = stats.norm.sf(np.abs(zvals))*2
pvals = pvals.reshape((-1,))
H, pvals_fdr, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')
pickle.dump({'pred_patterns':pred_patterns,
             'zvals':zvals,
             'pvals':pvals,
             'pvals':pvals_fdr},
             open(svfile, 'wb'))

In [ ]:
# ## gather results from all phases and trial-blocks
# ## run this cell after get results from each phase and trial-block
# ## i.e., when all files f'./sample_results/predict_pattern_{phase}_{mask}_{trial_block}block.pkl' are ready

# mask = 'fearnet' # feature to be used, 'fearnet' or 'wholebrain'
# all_pvals = []
# all_pvals_fdr = []
# all_zvals = []
# all_pred_patterns = []
# for cphs in ['cond', 'ext', 'recall']:
#     for tblk in [1,2,3,4]:
#         if cphs=='ext' and tblk>1:# only for trial-blocks with robust classification performance
#             continue
#         elif cphs=='recall' and tblk==2:
#             continue
#         elif cphs=='recall' and tblk==4:
#             continue
#         file = f'./sample_results/predict_pattern_{phase}_{mask}_{trial_block}block.pkl'
#         D = pickle.load(open(file, 'rb'))
#         zvals = D['zvals']
#         pvals = D['pvals']
#         pvals_fdr = D['pvals_fdr']
#         pred_patterns = D['pred_patterns']
        
#         all_zvals.append(zvals)
#         all_pvals.append(pvals)
#         all_pvals_fdr.append(pvals_fdr)
#         all_pred_patterns.append(pred_patterns)
# all_zvals = np.stack(all_zvals)
# all_pvals = np.stack(all_pvals)
# all_pvals_fdr = np.stack(all_pvals_fdr)
# all_pred_patterns = np.stack(all_pred_patterns)

# if mask=='wholebrain':
#     savefile = './sample_results/whole_brain_predictive_patterns.pkl'
# elif mask=='fearnet':
#     savefile = './sample_results/threat_circuit_predictive_patterns.pkl'
    
# pickle.dump({'all_zvals':all_zvals,
#              'all_pvals':all_pvals,
#              'all_pvals_fdr':all_pvals_fdr,
#              'all_pred_patterns':all_pred_patterns},
#              open(savefile, 'wb'))